# Data Wrangling in R (part 2)

## Table of Contents

- [Reshaping Data with tidyr](#resh)
- [Accessing databases with dbplyr](#db)
- [Accessing Web APIs](#api)

---
<a id='resh'></a>

## Reshaping Data with tidyr

One of the most common data wrangling challenges is adjusting how exactly rows and columns are used to represent data. [**tidyr ("tidy-er")**](https://tidyr.tidyverse.org/) is a package helping in structuring data frames to have the desired shape (transforming orientation) for visualization, running a statistical model or implementing a machine learning algorithm. **tidyr** helps in following the **principles of tidy data**. Tidy data is data where:

- Every column is variable.
- Every row is an observation.
- Every cell is a single value.


<img src="images/data-wide.png" alt="" style="width: 500px;"/>

The format is wide, because the price data is spread wide across multiple columns.

<img src="images/data-long.png" alt="" style="width: 500px;"/>

The format is long, because the price data has ist own column. This format includes duplicated cities and bands.

In [ ]:
# gather() - to move from wide format to long format 
#  you need to gather all of the prices into a single columns

# Reshape by gathering prices into a single feature
band_data_long <- gather(
    band_data_wide, # data frame to gather from
    # name for new column listing the gathered featu 
    # (will contain values of column names from the wide form)
    key = band, 
    # name for new column listing the gathered values 
    # (here will be all gathered values)
    value = price, 
    # columns to gather data from
    # (gather from all columns except city)
    -city 
)

<img src="images/data-tidyr-gather.png" alt="" style="width: 600px;"/>


In [ ]:
# spread() - from rows to columns, from long into wide format

# Reshape by spreading prices out among multiple features
price_by_band <- spread(
    band_data_long, # data frame to spread from
    key = city, # get new colum names from this column
    value = price # get values for the new columns from this column
)

<img src="images/data-tidyr-spread.png" alt="" style="width: 600px;"/>


In [ ]:
# Unite multiple columns into a single column
# unite()

# Separate a single column into multiple columns
# separate() 

---
<a id='db'></a>

## Accessing databases with dbplyr

**[dplyr](https://github.com/tidyverse/dbplyr)** is the database backend for [dplyr](https://dplyr.tidyverse.org/). It allows you to use remote database tables as if they are in-memory data frames by automatically converting dplyr code into SQL.

To learn more about why you might use dbplyr instead of writing SQL, see vignette("sql"). To learn more about the details of the SQL translation, see vignette("translation-verb") and vignette("translation-function").

In [2]:
install.packages("dbplyr")
library("DBI")
library("dplyr")


The downloaded binary packages are in
	/var/folders/qk/0l3zx9w11959pqp8tr5s_1780000gn/T//RtmpjGbWsk/downloaded_packages



Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




In [3]:
# To access an SQLite database 
install.packages("RSQLite") # once per machine 
library("RSQLite") # in each relevant script

# To access a Postgres database 
install.packages("RPostgreSQL") # once per machine 
library("RPostgreSQL") # in each relevant script

also installing the dependencies ‘bit’, ‘bit64’, ‘blob’





The downloaded binary packages are in
	/var/folders/qk/0l3zx9w11959pqp8tr5s_1780000gn/T//RtmpjGbWsk/downloaded_packages

The downloaded binary packages are in
	/var/folders/qk/0l3zx9w11959pqp8tr5s_1780000gn/T//RtmpjGbWsk/downloaded_packages


In [7]:
# Install and load the `dplyr`, `DBI`, and `RSQLite` packages for accessing
# databases
library("dplyr")
library("DBI")
library("RSQLite")

In [8]:
# Create a connection to the `Chinook_Sqlite.sqlite` file in the `data` folder
# Be sure to set your working directory!
db_connection <- dbConnect(SQLite(), dbname = "data/r/Chinook_Sqlite.sqlite")

In [9]:
# Use the `dbListTables()` function (passing in the connection) to get a list
# of tables in the database.
dbListTables(db_connection)

[1] "Album"         "Artist"        "Customer"      "Employee"     
 [5] "Genre"         "Invoice"       "InvoiceLine"   "MediaType"    
 [9] "Playlist"      "PlaylistTrack" "Track"

In [12]:
# Use the `tbl()`function to create a reference to the table of music genres.
# Print out the the table to confirm that you've accessed it.
genre_tbl <- tbl(db_connection, "Genre")
genre_tbl

# Source:   table<Genre> [?? x 2]
# Database: sqlite 3.29.0
#   [/Users/ksatola/Documents/git/Data-Science-Notes/data/r/Chinook_Sqlite.sqlite]
   GenreId Name              
     <int> <chr>             
 1       1 Rock              
 2       2 Jazz              
 3       3 Metal             
 4       4 Alternative & Punk
 5       5 Rock And Roll     
 6       6 Blues             
 7       7 Latin             
 8       8 Reggae            
 9       9 Pop               
10      10 Soundtrack        
# … with more rows

In [13]:
# Try to use `View()` to see the contents of the table. What happened?
View(genre_tbl)

ERROR: Error in View(genre_tbl): ‘View()’ not yet supported in the Jupyter R kernel


In [14]:
# Use the `collect()` function to actually load the genre table into memory
# as a data frame. View that data frame.
genre_df <- collect(genre_tbl)
View(genre_df)

ERROR: Error in View(genre_df): ‘View()’ not yet supported in the Jupyter R kernel


In [15]:
# Use dplyr's `count()` function to see how many rows are in the genre table
genre_tbl %>% count()

# Source:   lazy query [?? x 1]
# Database: sqlite 3.29.0
#   [/Users/ksatola/Documents/git/Data-Science-Notes/data/r/Chinook_Sqlite.sqlite]
      n
  <int>
1    25

In [16]:
# Use the `tbl()` function to create a reference the table with track data.
# Print out the the table to confirm that you've accessed it.
track_tbl <- tbl(db_connection, "Track")
print(track_tbl)

# Source:   table<Track> [?? x 9]
# Database: sqlite 3.29.0
#   [/Users/ksatola/Documents/git/Data-Science-Notes/data/r/Chinook_Sqlite.sqlite]
   TrackId Name  AlbumId MediaTypeId GenreId Composer Milliseconds  Bytes
     <int> <chr>   <int>       <int>   <int> <chr>           <int>  <int>
 1       1 For …       1           1       1 Angus Y…       343719 1.12e7
 2       2 Ball…       2           2       1 NA             342562 5.51e6
 3       3 Fast…       3           2       1 F. Balt…       230619 3.99e6
 4       4 Rest…       3           2       1 F. Balt…       252051 4.33e6
 5       5 Prin…       3           2       1 Deaffy …       375418 6.29e6
 6       6 Put …       1           1       1 Angus Y…       205662 6.71e6
 7       7 Let'…       1           1       1 Angus Y…       233926 7.64e6
 8       8 Inje…       1           1       1 Angus Y…       210834 6.85e6
 9       9 Snow…       1           1       1 Angus Y…       203102 6.60e6
10      10 Evil…       1           1       

In [17]:
# Use dplyr functions to query for a list of artists in descending order by
# popularity in the database (e.g., the artist with the most tracks at the top)
# - Start by filting for rows that have an artist listed (use `is.na()`), then
#   group rows by the artist and count them. Finally, arrange the results.
# - Use pipes to do this all as one statement without collecting the data into
#   memory!
popular_artists <- track_tbl %>%
  filter(is.na(Composer) == FALSE) %>%
  group_by(Composer) %>%
  count() %>%
  arrange(-n)
print(popular_artists)

# Source:     lazy query [?? x 2]
# Database:   sqlite 3.29.0
#   [/Users/ksatola/Documents/git/Data-Science-Notes/data/r/Chinook_Sqlite.sqlite]
# Groups:     Composer
# Ordered by: -n
   Composer                                           n
   <chr>                                          <int>
 1 Steve Harris                                      80
 2 U2                                                44
 3 Jagger/Richards                                   35
 4 Billy Corgan                                      31
 5 Kurt Cobain                                       26
 6 Bill Berry-Peter Buck-Mike Mills-Michael Stipe    25
 7 The Tea Party                                     24
 8 Chico Science                                     23
 9 Chris Cornell                                     23
10 Gilberto Gil                                      23
# … with more rows


In [19]:
# Use dplyr functions to query for the most popular _genre_ in the library.
# You will need to count the number of occurrences of each genre, and join the
# two tables together in order to also access the genre name.
# Collect the resulting data into memory in order to access the specific row of
# interest
genre_count_names_df <- genre_counts %>%
  group_by(GenreId) %>%
  count() %>%
  left_join(genre_tbl) %>%
  arrange(-count) %>%
  collect()
print(genre_count_names_df[1, "Name"])

ERROR: Error in eval(lhs, parent, parent): object 'genre_counts' not found


In [20]:
# Bonus: Query for a list of the most popular artist for each genre in the
# library (a "representative" artist for each).
# Consider using multiple grouping operations. Note that you can only filter
# for a `max()` value if you've collected the data into memory.
track_tbl %>%
  filter(is.na(Composer) == FALSE) %>%
  group_by(GenreId, Composer) %>%
  count() %>%
  left_join(genre_tbl) %>%
  select(Genre = Name, Composer, count = n) %>%
  collect() %>%
  group_by(Genre) %>%
  filter(count == max(count)) %>%
  arrange(-count)

Joining, by = "GenreId"



GenreId,Genre,Composer,count
<int>,<chr>,<chr>,<int>
1,Rock,U2,44
3,Metal,Steve Harris,36
4,Alternative & Punk,Billy Corgan,31
2,Jazz,Miles Davis,23
7,Latin,Chico Science,23
6,Blues,Chris Robinson/Rich Robinson,18
10,Soundtrack,"Brian Eno, Bono, Adam Clayton, The Edge & Larry Mullen Jnr.",14
16,World,João Suplicy,14
13,Heavy Metal,Steve Harris,13


In [21]:
# Remember to disconnect from the database once you are done with it!
dbDisconnect(db_connection)

---
<a id='api'></a>

## Accessing Web APIs

Access tokens are a lot like passwords; you will want to keep them secret and not share them with others. This means that you should not include them in any files you commit to git and push to GitHub. The best way to ensure the secrecy of access tokens in R is to create a separate script file in your repo (e.g., api_keys.R) that includes exactly one line, assigning the key to a variable:

In [22]:
# Store your API key from a web service in a variable 
# It should be in a separate file (e.g., `api_keys.R`) 
api_key <- "123456789abcdefg"

In [24]:
# In your "main" script (e.g., `my_script.R`) load your API key from another file
# (Make sure working directory is set before running the following code!)
source("data/r/api_keys.R") # load the script using a *relative path* 
print(api_key) # the key is now available!

[1] "123456789abcdefg"


Anyone else who runs the script will need to provide an `api_key` variable to access the API using that user’s own key. This practice keeps everyone’s account separate.
You can keep your `api_keys.R` file from being committed by including the filename in the `.gitignore` file in your repo; that will keep it from even possibly being committed with your code!

In [25]:
# The URI for the search/repositories endpoint of the GitHub API
https://api.github.com/search/repositories?q=dplyr&sort=forks

ERROR: Error in parse(text = x, srcfile = src): <text>:2:7: unexpected '/'
1: # The URI for the search/repositories endpoint of the GitHub API
2: https:/
         ^


Official quickstart guide for **[httr](https://cran.r-project.org/web/packages/httr/vignettes/quickstart.html)**, a package designed to map closely to the underlying http protocol.

In [26]:
install.packages("httr") # once per machine
library("httr") # in each relevant script


The downloaded binary packages are in
	/var/folders/qk/0l3zx9w11959pqp8tr5s_1780000gn/T//RtmpjGbWsk/downloaded_packages


In [27]:
# Make a GET request to the GitHub API's "search/" endpoint
# Request repositories that match the search "dplyr", and sort the results b 
url <- "https://api.github.com/search/repositories?q=dplyr&sort=forks"
response <- GET(url)

In [28]:
print(response)

Response [https://api.github.com/search/repositories?q=dplyr&sort=forks]
  Date: 2019-10-31 20:26
  Status: 200
  Content-Type: application/json; charset=utf-8
  Size: 181 kB
{
  "total_count": 980,
  "incomplete_results": false,
  "items": [
    {
      "id": 6427813,
      "node_id": "MDEwOlJlcG9zaXRvcnk2NDI3ODEz",
      "name": "dplyr",
      "full_name": "tidyverse/dplyr",
      "private": false,
...


In [29]:
# Restructure the previous request to make it easier to read and update.

# Make a GET request to the GitHub API's "search/" endpoint
# Request repositories that match the search "dplyr", sorted by forks

# Construct your `resource_uri` from a `base_uri` that you may reuse and an `endpoint`
base_uri <- "https://api.github.com" 
endpoint <- "/search/repositories" 
resource_uri <- paste0(base_uri, endpoint)

# Store any query parameters you want to use in a list
query_params <- list(q = "dplyr", sort = "forks")

# Make your request, specifying the query parameters via the `query` argumen
response <- GET(resource_uri, query = query_params)

In [30]:
response

Response [https://api.github.com/search/repositories?q=dplyr&sort=forks]
  Date: 2019-10-31 20:29
  Status: 200
  Content-Type: application/json; charset=utf-8
  Size: 181 kB
{
  "total_count": 980,
  "incomplete_results": false,
  "items": [
    {
      "id": 6427813,
      "node_id": "MDEwOlJlcG9zaXRvcnk2NDI3ODEz",
      "name": "dplyr",
      "full_name": "tidyverse/dplyr",
      "private": false,
...

In [31]:
# Extract content from `response`, as a text string
response_text <- content(response, type = "text")

No encoding supplied: defaulting to UTF-8.



While the **httr** package is able to parse the JSON body of a response into a list, it doesn’t do a very clean job of it (particularly for complex data structures). A more effective solution for transforming JSON data is to use the **[jsonlite package](https://cran.r-project.org/web/packages/jsonlite/jsonlite.pdf)**. This package provides helpful methods to convert JSON data into R data, and is particularly well suited for converting content into data frames.

In [33]:
install.packages("jsonlite") # once per machine 
library("jsonlite") # in each relevant script


The downloaded binary packages are in
	/var/folders/qk/0l3zx9w11959pqp8tr5s_1780000gn/T//RtmpjGbWsk/downloaded_packages


In [35]:
# Make a request to a given `uri` with a set of `query_params` 
# Then extract and parse the results

# Make the request
response <- GET(resource_uri, query = query_params) 

# Extract the content of the response
response_text <- content(response, "text")

# Convert the JSON string to a list
response_data <- fromJSON(response_text)

In [41]:
# Use various methods to explore and extract information from API results 

# Check: is it a data frame already?
is.data.frame(response_data) # FALSE

[1] FALSE

In [42]:
# Inspect the data!
str(response_data) # view as a formatted string
names(response_data) # "href" "items" "limit" "next" "offset" "previous" "total"

List of 3
 $ total_count       : int 981
 $ incomplete_results: logi FALSE
 $ items             :'data.frame':	30 obs. of  74 variables:
  ..$ id               : int [1:30] 6427813 67845042 59305491 84520584 24485567 66818484 55175084 128841385 76831311 77705369 ...
  ..$ node_id          : chr [1:30] "MDEwOlJlcG9zaXRvcnk2NDI3ODEz" "MDEwOlJlcG9zaXRvcnk2Nzg0NTA0Mg==" "MDEwOlJlcG9zaXRvcnk1OTMwNTQ5MQ==" "MDEwOlJlcG9zaXRvcnk4NDUyMDU4NA==" ...
  ..$ name             : chr [1:30] "dplyr" "m9-dplyr" "sparklyr" "Data-Analysis-with-R" ...
  ..$ full_name        : chr [1:30] "tidyverse/dplyr" "INFO-201/m9-dplyr" "rstudio/sparklyr" "susanli2016/Data-Analysis-with-R" ...
  ..$ private          : logi [1:30] FALSE FALSE FALSE FALSE FALSE FALSE ...
  ..$ owner            :'data.frame':	30 obs. of  18 variables:
  .. ..$ login              : chr [1:30] "tidyverse" "INFO-201" "rstudio" "susanli2016" ...
  .. ..$ id                 : int [1:30] 22032646 19807570 513560 24217243 6509492 34749559 1250583

[1] "total_count"        "incomplete_results" "items"

In [43]:
# Looking at the JSON data itself (e.g., in the browser), 
# `items` is the key that contains the value you want

# Extract the (useful) data
items <- response_data$items # extract from the list 
is.data.frame(items) # TRUE; you can work with that!

[1] TRUE

In [49]:
#print(items)

In [52]:
# A demonstration of the structure of "nested" data frames 
# Create a `people` data frame with a `names` column
people <- data.frame(names = c("Ed", "Jessica", "Keagan"))

# Create a data frame of favorites with two columns
favorites <- data.frame(
    food = c("Pizza", "Pasta", "salad"),
    music = c("Bluegrass", "Indie", "Electronic")
)

# Store the second data frame as a column of the first -- A BAD IDEA 
people$favorites <- favorites # the `favorites` column is a data frame!

# This prints nicely, but is misleading
print(people) 

# Despite what RStudio prints, there is not actually a column `favorites.foo` 
people$favorites.food # NULL

# Access the `food` column of the data frame stored in `people$favorites` 
people$favorites$food # [1] Pizza Pasta salad

    names favorites.food favorites.music
1      Ed          Pizza       Bluegrass
2 Jessica          Pasta           Indie
3  Keagan          salad      Electronic


NULL

[1] Pizza Pasta salad
Levels: Pasta Pizza salad

Nested data frames make it hard to work with the data using previously established techniques and syntax. Luckily, the jsonlite package provides a helpful function for addressing this issue, called **flatten()**. This function takes the columns of each nested data frame and converts them into appropriately named columns in the “outer” data frame.

In [53]:
# Use `flatten()` to format nested data frames
people <- flatten(people)
people$favorites.food # this just got created! Woo!

[1] Pizza Pasta salad
Levels: Pasta Pizza salad

In [ ]:
---
<a id='data'></a>